In [ ]:
import argparse
from webcam_utils import realtime_emotions
from prediction_utils import prediction_path

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


In [5]:
# for running realtime emotion detection
import easydict

def emotion_detection(img_str):
    return prediction_path(img_str)
    

In [6]:
from flask import Flask, request, jsonify
app = Flask(__name__)
host = 'localhost'
port = 5000

@app.route('/emotion', methods = ['POST'])
def analysisEmotiom():
    imageData = request.get_json()['image']
    return jsonify({'result': emotion_detection(imageData)})

@app.route('/recommendations', methods = ['POST'])
def analysisEmotionAndReturnRecommends():
    imageData = request.get_json()['image']
    return jsonify({'result': 'test'})

@app.route('/wakeup', methods = ['GET'])
def wakeup():
    return jsonify({'status': 'open'}) 

app.run(host=host, port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://localhost:5000/ (Press CTRL+C to quit)
[2020-11-23 01:19:04,787] ERROR in app: Exception on /emotion [POST]
Traceback (most recent call last):
  File "C:\Users\gkwogus\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\gkwogus\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\gkwogus\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\gkwogus\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\gkwogus\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\gkwogus\Anaconda3\lib